# 🔥 IEEE-CIS Fraud Detection — Model Training

Full pipeline: Load → Preprocess → Feature Engineer → Train (LGB/XGB/CAT) → Evaluate

### Data Sources Added
| Dataset | Kaggle slug |
|---------|-------------|
| IEEE fraud data | `muhakabartay/yourallmodelsdata` |
| Source code | `aryyaaaaa/fraud-detection-src` |

In [ ]:
import os, sys, shutil, gc, time, warnings
warnings.filterwarnings('ignore')
import numpy as np, pandas as pd
import matplotlib.pyplot as plt, seaborn as sns
import logging

logging.basicConfig(level=logging.INFO, format='%(message)s')

# Matplotlib style — fall back gracefully
try:
    plt.style.use('seaborn-v0_8-whitegrid')
except OSError:
    plt.style.use('seaborn-whitegrid')
plt.rcParams.update({'figure.figsize': [14, 6], 'figure.dpi': 110})

OUT = '/kaggle/working'

# ── AUTO-DETECT DATA PATH ─────────────────────────
DATA_CANDIDATES = [
    '/kaggle/input/yourallmodelsdata',
    '/kaggle/input/datasets/muhakabartay/yourallmodelsdata',
]
INPUT = None
for p in DATA_CANDIDATES:
    if os.path.isdir(p):
        INPUT = p
        break
if INPUT is None:
    for d in os.listdir('/kaggle/input'):
        full = f'/kaggle/input/{d}'
        if os.path.isdir(full) and os.path.exists(f'{full}/train_transaction.csv'):
            INPUT = full
            break
assert INPUT is not None, (
    'Could not find train_transaction.csv. '
    'Add the IEEE-CIS dataset via \"Add Data\".'
)
print(f'✅ Data path: {INPUT}')
print(f'   Files: {os.listdir(INPUT)}')

# ── AUTO-DETECT SRC PATH ──────────────────────────
SRC_CANDIDATES = [
    '/kaggle/input/fraud-detection-src',
    '/kaggle/input/datasets/aryyaaaaa/fraud-detection-src',
]
SRC_INPUT = None
for p in SRC_CANDIDATES:
    if os.path.isdir(p):
        SRC_INPUT = p
        break
if SRC_INPUT is None:
    for d in os.listdir('/kaggle/input'):
        full = f'/kaggle/input/{d}'
        # Look for a directory that contains src/ or data/ subpackage
        if os.path.isdir(full) and (
            os.path.isdir(f'{full}/src') or os.path.isdir(f'{full}/data')
        ):
            if full != INPUT:  # Don't confuse with the data dataset
                SRC_INPUT = full
                break
assert SRC_INPUT is not None, (
    'Could not find src modules. '
    'Add the fraud-detection-src dataset via \"Add Data\".'
)
print(f'✅ Src  path: {SRC_INPUT}')
print(f'   Contents: {os.listdir(SRC_INPUT)}')

# ── Copy src/ into /kaggle/working for imports ────
dst = f'{OUT}/src'
if os.path.exists(dst):
    shutil.rmtree(dst)

if os.path.isdir(f'{SRC_INPUT}/src'):
    shutil.copytree(f'{SRC_INPUT}/src', dst)
    print(f'✅ Copied {SRC_INPUT}/src → {dst}')
elif os.path.isdir(f'{SRC_INPUT}/data') and os.path.isdir(f'{SRC_INPUT}/features'):
    # The dataset root IS the src package contents
    shutil.copytree(SRC_INPUT, dst)
    print(f'✅ Copied {SRC_INPUT} (root) → {dst}')
else:
    raise FileNotFoundError(
        f'Cannot find src/ inside {SRC_INPUT}. '
        f'Contents: {os.listdir(SRC_INPUT)}'
    )

sys.path.insert(0, OUT)

# Quick smoke-test import
from src.data.preprocessor import DataPreprocessor
print(f'✅ Import smoke-test passed')

In [ ]:
!pip install -q lightgbm xgboost catboost 2>/dev/null
print('✅ Dependencies ready')

---
## 1 · Load Data

In [ ]:
t0 = time.time()
print('Loading train_transaction.csv …')
train_txn = pd.read_csv(f'{INPUT}/train_transaction.csv')
print(f'  → {len(train_txn):,} rows')

print('Loading train_identity.csv …')
train_id = pd.read_csv(f'{INPUT}/train_identity.csv')
print(f'  → {len(train_id):,} rows')

print('Merging …')
df = train_txn.merge(train_id, on='TransactionID', how='left')
del train_txn, train_id; gc.collect()

print(f'\n📦 Shape: {df.shape}  Fraud: {df.isFraud.mean():.4%}  '
      f'Mem: {df.memory_usage(deep=True).sum()/1e9:.2f} GB  '
      f'Time: {time.time()-t0:.0f}s')

---
## 2 · Preprocess

In [ ]:
from src.data.preprocessor import DataPreprocessor

preprocessor = DataPreprocessor()
t0 = time.time()

print('Cleaning …')
df_clean = preprocessor.clean_data(df)
del df; gc.collect()

print('Splitting (time-based) …')
train_df, val_df, test_df = preprocessor.time_based_split(df_clean)
del df_clean; gc.collect()

for name, d in [('Train', train_df), ('Val', val_df), ('Test', test_df)]:
    print(f'  {name:5s}: {len(d):>8,}  fraud={d.isFraud.mean():.4%}')
print(f'⏱ {time.time()-t0:.0f}s')

---
## 3 · Feature Engineering

In [ ]:
from src.features.engineer import FraudFeatureEngineer

engineer = FraudFeatureEngineer(lookback_windows=[1, 7, 30])
t0 = time.time()

print('Train features …')
train_feat = engineer.fit_transform(train_df, train_df=train_df)
print(f'  → {train_feat.shape}')

print('Val features …')
val_feat = engineer.fit_transform(val_df, train_df=train_df)
print(f'  → {val_feat.shape}')

print('Test features …')
test_feat = engineer.fit_transform(test_df, train_df=train_df)
print(f'  → {test_feat.shape}')

del train_df, val_df, test_df; gc.collect()
print(f'\n✅ Done in {time.time()-t0:.0f}s  Features: {train_feat.shape[1]}')

In [ ]:
# Build X/y — keep only numeric features
EXCLUDE = {'TransactionID', 'isFraud', 'TransactionDT',
           'transaction_datetime', 'FraudLabel', 'datetime',
           'card1', 'card2'}

NUMERIC_DTYPES = {'int64','float64','int32','float32','int16','float16','uint8'}

feat_cols = [c for c in train_feat.columns
             if c not in EXCLUDE
             and str(train_feat[c].dtype) in NUMERIC_DTYPES]

# Ensure val/test have same columns (drop any that don't exist)
feat_cols = [c for c in feat_cols
             if c in val_feat.columns and c in test_feat.columns]

X_train = train_feat[feat_cols].fillna(-999).astype('float32')
y_train = train_feat['isFraud'].values

X_val = val_feat[feat_cols].fillna(-999).astype('float32')
y_val = val_feat['isFraud'].values

X_test = test_feat[feat_cols].fillna(-999).astype('float32')
y_test = test_feat['isFraud'].values

del train_feat, val_feat, test_feat; gc.collect()
print(f'Features: {len(feat_cols)}  X_train: {X_train.shape}  '
      f'Mem: {X_train.memory_usage(deep=True).sum()/1e9:.2f} GB')

---
## 4 · Train LightGBM

In [ ]:
from src.models.lightgbm_model import LightGBMFraudDetector

lgb_m = LightGBMFraudDetector()
t0 = time.time()
lgb_r = lgb_m.train(X_train, y_train, X_val, y_val,
                    num_boost_round=1000, early_stopping_rounds=50)
print(f'⏱ {time.time()-t0:.0f}s')
lgb_m.save(f'{OUT}/lightgbm_detector.pkl')
print('💾 Saved')

## 5 · Train XGBoost

In [ ]:
from src.models.xgboost_model import XGBoostFraudDetector

xgb_m = XGBoostFraudDetector()
t0 = time.time()
xgb_r = xgb_m.train(X_train, y_train, X_val, y_val,
                    num_boost_round=1000, early_stopping_rounds=50)
print(f'⏱ {time.time()-t0:.0f}s')
xgb_m.save(f'{OUT}/xgboost_detector.pkl')
print('💾 Saved')

## 6 · Train CatBoost

In [ ]:
from src.models.catboost_model import CatBoostFraudDetector

cat_m = CatBoostFraudDetector()
cat_m.params['iterations'] = 1000
cat_m.params['verbose'] = 100
t0 = time.time()
cat_r = cat_m.train(X_train, y_train, X_val, y_val)
print(f'⏱ {time.time()-t0:.0f}s')
cat_m.save(f'{OUT}/catboost_detector.cbm')
print('💾 Saved')

---
## 7 · Evaluation

In [ ]:
from src.evaluation.metrics import evaluate_fraud_model, compare_models, find_optimal_threshold

lgb_p = lgb_m.predict(X_test)
xgb_p = xgb_m.predict(X_test)
cat_p = cat_m.predict(X_test)

preds = {'LightGBM': lgb_p, 'XGBoost': xgb_p, 'CatBoost': cat_p}
comp = compare_models(y_test, preds)

print('\n' + '='*60)
print('  MODEL COMPARISON (Test Set)')
print('='*60)
print(comp.to_string(index=False))
print('='*60)
comp.to_csv(f'{OUT}/model_comparison.csv', index=False)

In [ ]:
# Best model details
best = comp.iloc[0]['Model']
print(f'\n🏆 Best model: {best}\n')
_ = evaluate_fraud_model(y_test, preds[best])

# find_optimal_threshold returns (threshold, metrics_dict)
opt_thresh, opt_metrics = find_optimal_threshold(y_test, preds[best], metric='f1')
print(f'\nOptimal threshold: {opt_thresh:.4f}')
print(f'  F1       : {opt_metrics["f1"]:.4f}')
print(f'  Precision: {opt_metrics["precision"]:.4f}')
print(f'  Recall   : {opt_metrics["recall"]:.4f}')

In [ ]:
# ROC + PR curves
from sklearn.metrics import roc_curve, precision_recall_curve, roc_auc_score, average_precision_score

pal = {'LightGBM': '#2ecc71', 'XGBoost': '#3498db', 'CatBoost': '#e74c3c'}
fig, ax = plt.subplots(1, 2, figsize=(16, 6))

for n, p in preds.items():
    c = pal[n]
    fpr, tpr, _ = roc_curve(y_test, p)
    ax[0].plot(fpr, tpr, color=c, lw=2,
               label=f'{n} ({roc_auc_score(y_test, p):.4f})')
    pre, rec, _ = precision_recall_curve(y_test, p)
    ax[1].plot(rec, pre, color=c, lw=2,
               label=f'{n} ({average_precision_score(y_test, p):.4f})')

ax[0].plot([0,1], [0,1], 'k--', alpha=0.3)
ax[0].set(title='ROC Curve', xlabel='FPR', ylabel='TPR')
ax[0].legend(loc='lower right')
ax[1].set(title='Precision-Recall Curve', xlabel='Recall', ylabel='Precision')
ax[1].legend(loc='lower left')

plt.tight_layout()
plt.savefig(f'{OUT}/roc_pr.png', dpi=200, bbox_inches='tight')
plt.show()

In [ ]:
# Feature importance (LightGBM top 25)
top = lgb_m.get_feature_importance(25)
fig, ax = plt.subplots(figsize=(10, 8))
ax.barh(top['feature'].values[::-1], top['importance'].values[::-1], color='#3498db')
ax.set_title('LightGBM — Top 25 Features', fontweight='bold')
plt.tight_layout()
plt.savefig(f'{OUT}/feature_importance.png', dpi=200, bbox_inches='tight')
plt.show()
top.to_csv(f'{OUT}/feature_importance.csv', index=False)

---
## 8 · Summary

In [ ]:
print('='*60 + '\n  📊 TRAINING SUMMARY\n' + '='*60)
print(f'Dataset   : {X_train.shape[0] + X_val.shape[0] + X_test.shape[0]:,} txns × {len(feat_cols)} features')
print(f'Fraud rate: ~{y_train.mean():.2%}\n')
print(comp.to_string(index=False))
print(f'\nOutputs (download from Output tab):')
for f in sorted(os.listdir(OUT)):
    fp = f'{OUT}/{f}'
    if os.path.isfile(fp) and not f.startswith('.'):
        sz = os.path.getsize(fp) / 1e6
        print(f'  {f:40s} {sz:>7.1f} MB')
print('\n✅ Download models → place in local models/ → run API')
print('   uvicorn api.main:app --reload --port 8000')